In [7]:
#instalar docker
!apt-get -qq install docker.io

In [8]:
# Check version.
!docker --version

Docker version 24.0.5, build 24.0.5-0ubuntu1~22.04.1


In [13]:
%%shell
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker info
docker network ls
docker pull hello-world
docker images
docker run hello-world
kill $(jobs -p)

+ dockerd -b none --iptables=0 -l warn
++ seq 5
+ for i in $(seq 5)
+ '[' '!' -S /var/run/docker.sock ']'
+ sleep 2
WARN[2024-05-02T14:25:35.885642146Z] failed to load plugin io.containerd.snapshotter.v1.devmapper  error="devmapper not configured"
WARN[2024-05-02T14:25:35.885864830Z] could not use snapshotter devmapper in metadata plugin  error="devmapper not configured"
ERRO[2024-05-02T14:25:35.922897958Z] failed to mount overlay: invalid argument     storage-driver=overlay2
ERRO[2024-05-02T14:25:35.923038459Z] exec: "fuse-overlayfs": executable file not found in $PATH  storage-driver=fuse-overlayfs
WARN[2024-05-02T14:25:35.963055384Z] Failed to disable IPv6 on all interfaces on network namespace "/var/run/docker/netns/4a5010508663": failed to disable IPv6 forwarding for container's interface all: open /proc/sys/net/ipv6/conf/all/disable_ipv6: read-only file system 
ERRO[2024-05-02T14:25:35.963229687Z] failed to create osl sandbox while trying to restore sandbox 4a50105 for cleanup: o

In [11]:
# Establezca la ruta al conjunto de datos.
# Aquí descargamos y utilizamos un conjunto de datos que contiene 354 imágenes que muestran ropa.
# Si desea utilizar su propio conjunto de datos, simplemente establezca dataset_path en él.
dataset_path = "dataset_clothing_images"
!git clone https://github.com/lightly-ai/dataset_clothing_images.git {dataset_path}

fatal: destination path 'dataset_clothing_images' already exists and is not an empty directory.


In [5]:
# Install the Lightly Python Client
!pip3 install lightly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 741.2/741.2 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 34.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2

Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?
docker: Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?.
See 'docker run --help'.


In [12]:
# Instale Lightly Worker y realice una verificación rápida de cordura
# Si estos comandos fallan, siga nuestra guía de instalación de Docker en https://docs.lightly.ai/docs/install-lightly#docker
!docker pull lightly/worker:latest
!docker run --shm-size="1024m" --rm -it lightly/worker:latest sanity_check=True

Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?
docker: Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?.
See 'docker run --help'.


In [ ]:
# Schedule a run
from lightly.api import ApiWorkflowClient
from lightly.openapi_generated.swagger_client import DatasetType, DatasourcePurpose

lightly_token = "CHANGE_ME" # Copy the API_TOKEN from https://app.lightly.ai/preferences

# Create the Lightly client to connect to the API.
client = ApiWorkflowClient(token=lightly_token)

# Register the Lightly Worker.
worker_id = client.register_compute_worker(name="clothing-worker")

# Create the dataset on the Lightly Platform.
# If a dataset with this name already exists, you have to change the name.
client.create_dataset(
    dataset_name="clothing-small",
    dataset_type=DatasetType.IMAGES,
)

# Configure the datasource.
client.set_local_config(
    purpose=DatasourcePurpose.INPUT,
)
client.set_local_config(
    purpose=DatasourcePurpose.LIGHTLY,
)

# Schedule a run on the dataset to select 50 samples.
scheduled_run_id = client.schedule_compute_worker_run(
    worker_config={"shutdown_when_job_finished": True},
    selection_config={
        "n_samples": 50,
        "strategies": [
            {"input": {"type": "EMBEDDINGS"}, "strategy": {"type": "DIVERSITY"}}
        ],
    },
)

In [6]:
## tener instaaldo previamete Lightly
!docker pull lightly/worker:latest
!docker run --shm-size="1024m" --rm -it lightly/worker:latest sanity_check=True

Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?
docker: Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?.
See 'docker run --help'.


In [1]:
!pip3 install lightly
!pip3 install --upgrade lightly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 741.2/741.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 24.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.

In [5]:
#INSTALAR DOCKER
%%shell
sudo curl -fsSL https://get.docker.com/ | sh
sudo usermod -aG docker root
sudo docker run hello-world --priveleged
sudo dockerd --debug

# Executing docker install script, commit: e5543d473431b782227f8908005543bb4389b8de

If you already have Docker installed, this script can cause trouble, which is
why we're displaying this warning and provide the opportunity to cancel the
installation.

If you installed the current Docker package using this script and are using it
again to update Docker, you can safely ignore this message.

You may press Ctrl+C now to abort this script.
+ sleep 20
+ sh -c apt-get update -qq >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get install -y -qq apt-transport-https ca-certificates curl >/dev/null
+ sh -c install -m 0755 -d /etc/apt/keyrings
+ sh -c curl -fsSL "https://download.docker.com/linux/ubuntu/gpg" | gpg --dearmor --yes -o /etc/apt/keyrings/docker.gpg
+ sh -c chmod a+r /etc/apt/keyrings/docker.gpg
+ sh -c echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/docker.gpg] https://download.docker.com/linux/ubuntu jammy stable" > /etc/apt/sources.list.d/docker.list
+ sh -c apt-get updat

CalledProcessError: Command 'sudo curl -fsSL https://get.docker.com/ | sh
sudo usermod -aG docker root
sudo docker run hello-world --priveleged
sudo dockerd --debug
' returned non-zero exit status 1.

In [ ]:
%%shell

sudo apt update -qq

sudo apt install apt-transport-https ca-certificates curl software-properties-common -qq

curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -

sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu bionic stable"

sudo apt update -qq

sudo apt install docker-ce

docker

45 packages can be upgraded. Run 'apt list --upgradable' to see them.
ca-certificates is already the newest version (20230311ubuntu0.22.04.1).
curl is already the newest version (7.81.0-1ubuntu1.16).
software-properties-common is already the newest version (0.99.22.9).
apt-transport-https is already the newest version (2.4.12).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
OK
Repository: 'deb [arch=amd64] https://download.docker.com/linux/ubuntu bionic stable'
Description:
Archive for codename: bionic components: stable
More info: https://download.docker.com/linux/ubuntu
Adding repository.
Press [ENTER] to continue or Ctrl-c to cancel.Aborted.


CalledProcessError: Command '
sudo apt update -qq
    
sudo apt install apt-transport-https ca-certificates curl software-properties-common -qq
    
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -
    
sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu bionic stable"
    
sudo apt update -qq
    
sudo apt install docker-ce
    
docker
' died with <Signals.SIGINT: 2>.

In [ ]:
!pip install udocker
!udocker --allow-root install

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 2.6 MB/s eta 0:00:00
Info: creating repo: /root/.udocker
Info: udocker command line interface 1.3.16
Info: searching for udockertools >= 1.2.11
Info: installing udockertools 1.2.11
Info: installation of udockertools successful


In [ ]:
!udocker --allow-root run  \
  <image-repository>:tag

/bin/bash: line 1: image-repository: No such file or directory


In [ ]:
!udocker --allow-root ps

CONTAINER ID                         P M NAMES              IMAGE               


In [ ]:
!docker pull lightly/worker:latest
!docker run --shm-size="1024m" --rm -it lightly/worker:latest sanity_check=True

Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?
docker: Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?.
See 'docker run --help'.


In [ ]:
!docker run --shm-size "1024m" --gpus all --rm -it \
    -v /home/user/project_xyz/input:/input_mount:ro \
    -v /home/user/project_xyz/lightly:/lightly_mount \
    -e LIGHTLY_TOKEN='a07442857e87e75f4d5d71ed5c3e6572203db3916bafcac7' \
    -e LIGHTLY_WORKER_ID='662954bf92e9f46a03d7165a' \
    lightly/worker:latest

docker: Cannot connect to the Docker daemon at unix:///var/run/docker.sock. Is the docker daemon running?.
See 'docker run --help'.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ruta_carpeta_drive = '/content/drive/MyDrive/ALYOLO/datasets'

# Opcional: Copiar la carpeta a tu entorno local de Colab para mejorar el rendimiento
# Puedes omitir este paso si prefieres trabajar directamente desde Drive,
# pero ten en cuenta que puede ser más lento para operaciones de lectura/escritura intensivas.
!cp -r "{ruta_carpeta_drive}" ./datasets

In [ ]:
!docker pull lightly/worker:latest

/bin/bash: line 1: docker: command not found


In [ ]:
from pathlib import Path
from datetime import datetime
from lightly.api import ApiWorkflowClient
from lightly.openapi_generated.swagger_client import DatasetType, DatasourcePurpose

###### CHANGE THESE 2 VARIABLES
DATASET_PATH = Path("CHANGE_ME")  # e.g., Path("/path/to/images") or Path("clothing_dataset")
LIGHTLY_TOKEN = "a07442857e87e75f4d5d71ed5c3e6572203db3916bafcac7"  # Copy from https://app.lightly.ai/preferences
######

assert DATASET_PATH.exists(), f"Dataset path {DATASET_PATH} does not exist."

# Create the Lightly client to connect to the API.
client = ApiWorkflowClient(token=LIGHTLY_TOKEN)

# Register the Lightly Worker.
worker_id = client.register_compute_worker(name="first-worker")

# Create the dataset on the Lightly Platform.
# See our guide for more details and options:
# https://docs.lightly.ai/docs/set-up-your-first-dataset
client.create_dataset(
    dataset_name=f"first_dataset__{datetime.now().strftime('%Y_%m_%d__%H_%M_%S')}",
    dataset_type=DatasetType.IMAGES,
)

# Configure the datasources.
# See our guide for more details and options:
# https://docs.lightly.ai/docs/set-up-your-first-dataset
client.set_local_config(purpose=DatasourcePurpose.INPUT)
client.set_local_config(purpose=DatasourcePurpose.LIGHTLY)

# Schedule a run on the dataset to select 50 diverse samples.
# See our guide for more details and options:
# https://docs.lightly.ai/docs/run-your-first-selection
scheduled_run_id = client.schedule_compute_worker_run(
    worker_config={"shutdown_when_job_finished": True},
    selection_config={
        "n_samples": 50,
        "strategies": [
            {"input": {"type": "EMBEDDINGS"}, "strategy": {"type": "DIVERSITY"}}
        ],
    },
)

# Print the next commands
print(
    f"\nDocker Run command: \n"
    f"\033[7m"
    f"docker run --shm-size='1024m' --rm -it \\\n"
    f"\t-v '{DATASET_PATH.absolute()}':/input_mount:ro \\\n"
    f"\t-v '{Path('lightly').absolute()}':/lightly_mount \\\n"
    f"\t-e LIGHTLY_TOKEN={LIGHTLY_TOKEN} \\\n"
    f"\t-e LIGHTLY_WORKER_ID={worker_id} \\\n"
    "\tlightly/worker:latest\n"
    f"\033[0m"
)
print(
    "\nLightly Serve command:\n"
    f"\033[7m"
    f"lightly-serve input_mount='{DATASET_PATH.absolute()}' "
    f"lightly_mount='{Path('lightly').absolute()}'\n"
    f"\033[0m"
)


In [ ]:
from ultralytics.utils.downloads import GITHUB_ASSETS_STEMS
from ultralytics import YOLO, settings

model = YOLO("yolov8l.pt")

model.train(
    data="/content/datasets/custom_dataset.yaml",
    flipud=0.5,
    epochs=50,
    seed=0,
    batch=8,
    imgsz=960,
    name="random-200",
)  # train the model

Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/content/datasets/custom_dataset.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=960, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=random-2002, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf

train: Scanning /content/datasets/train/labels.cache... 843 images, 3 backgrounds, 0 corrupt: 100%|██████████| 843/843 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/datasets/valid/labels.cache... 302 images, 0 backgrounds, 0 corrupt: 100%|██████████| 302/302 [00:00<?, ?it/s]


Plotting labels to runs/detect/random-2002/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 960 train, 960 val
Using 0 dataloader workers
Logging results to runs/detect/random-2002
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/106 [00:00<?, ?it/s]

In [ ]:
yolo detect val model=runs/detect/random-200/weights/best.pt \
    data=data/custom_dataset.yaml \
    batch=8 imgsz=960 split=val

In [ ]:
# Código de ejemplo para sincronizar el conjunto de datos Lightly para seleccionar aleatoriamente 400
# imágenes
from lightly.api import ApiWorkflowClient

# Cree el cliente Lightly para conectarse a la API.
client = ApiWorkflowClient(token="a07442857e87e75f4d5d71ed5c3e6572203db3916bafcac7", dataset_id="Lightly-0eb087aeb548a6c5357f4f92e42b9991405eede39ec7d37717cff24bfa30c5fb")

# Descargue los nombres de archivos seleccionados de Lightly
filenames = client.export_filenames_by_tag_name(
    tag_name="initial-tag"  # name of the tag in the dataset
)

# Necesitamos reformatear un poco las cadenas.
filenames = filenames.split('\n')
filenames = ['all_fields_lincolnbeet/all/' + f + '\n' for f in filenames]

# Cargamos la configuración de YOLO para el conjunto de entrenamiento.
with open('all_fields_lincolnbeet/all_fields_lincolnbeet_train_.txt', 'r') as f:
    data = f.readlines()

# Solo conservamos los nombres de archivos de entrenamiento que han sido seleccionados por Lightly
data_filtered = list(filter(lambda x: x in filenames, data))

# Cree un nuevo archivo de texto con los datos ligeramente seleccionados
with open('all_fields_lincolnbeet/all_fields_lincolnbeet_train_random-400.txt', 'w') as f:
    f.writelines(data_filtered)

In [ ]:
scheduled_run_id = client.schedule_compute_worker_run(
    worker_config={
    },
    selection_config={
        "n_samples": 400,
        "strategies": [
            {
                "input": {
                    "type": "RANDOM",
                    "random_seed": 42, # optional, for reproducibility
                },
                "strategy": {
                    "type": "WEIGHTS",
                }
            }
        ],
    }
)

In [ ]:
scheduled_run_id = client.schedule_compute_worker_run(
    worker_config={
        "enable_training": True,
        "training": {
                "task_name": "yolov8-random-200-detection",
        },
    },
    selection_config={
        "n_samples": 400,
        "strategies": [
            {
                # strategy to find diverse objects
                "input": {
                    "type": "EMBEDDINGS",
                    "task": "yolov8-random-200-detection",
                },
                "strategy": {
                    "type": "DIVERSITY",
                },
            },
            {
                # strategy to balance the class ratios
                "input": {
                    "type": "PREDICTIONS",
                    "name": "CLASS_DISTRIBUTION",
                    "task": "yolov8-random-200-detection",
                },
                "strategy": {
                    "type": "BALANCE",
                    "target": {
                        "sugar beet": 0.3,
                        "weed": 0.7
                    },
                },
            },
            {
                # strategy to prioritize images with more objects (frequency)
                "input": {
                    "type": "SCORES",
                    "task": "yolov8-random-200-detection",
                    "score": "object_frequency",
                },
                "strategy": {"type": "WEIGHTS"},
            },
            {
                # strategy to use prediction score (Active Learning)
                "input": {
                    "type": "SCORES",
                    "task": "yolov8-random-200-detection",
                    "score": "objectness_least_confidence",
                },
                "strategy": {"type": "WEIGHTS"},
            },
        ],
    },
    lightly_config={
        "trainer": {
            "max_epochs": 1,
            "precision": 16
        },
        "loader": {
            "batch_size": 128
        },
        "collate": {
            "vf_prob": 0.5, # enable vertical flip
            "rr_prob": 0.5, # enable random rotation
        }
    },
)